In [48]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader
import pandas as pd
import math
import random
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

In [49]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def set_seed(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

def build_sequence_dataset(df, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(df) - seq_length):
        _x = df.iloc[i:i + seq_length].values
        _y = df.iloc[i + seq_length]['Value']
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size = x.size(0)

        h0, c0 = self.init_hidden(batch_size, x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

    def init_hidden(self, batch_size, device):
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        return h0, c0


def train(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def validate_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    actuals= []
    predictions = []

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target.view(-1, 1))
            total_loss += loss.item()

            actuals.extend(target.squeeze(1).tolist())
            predictions.extend(output.squeeze(1).tolist())

    avg_loss = total_loss / len(test_loader)
    return avg_loss, actuals, predictions

In [51]:
df_ppg = pd.read_csv('/home/youngwoo/export_cda.csv')
df_ppg = df_ppg[df_ppg['MeasureName'] == 'Heart rate']['Value'].to_frame()

seed_value = 42
set_seed(seed_value)

num_output = 1
num_hidden = 10
num_features = 1

seq_length = 20
batch_size = 32
learning_rate = 0.001

scaler = StandardScaler()
ppg_scaled_arr = scaler.fit_transform(df_ppg)
df_ppg_scaled = pd.DataFrame(ppg_scaled_arr, columns=['Value']) 

sequence_dataX, sequence_dataY = build_sequence_dataset(df_ppg_scaled, seq_length)

train_X, test_X, train_Y, test_Y = train_test_split(
    sequence_dataX, sequence_dataY, test_size=0.2, shuffle=False
)

train_X_tensor = torch.tensor(train_X, dtype=torch.float32)
train_Y_tensor = torch.tensor(train_Y.reshape(-1, 1), dtype=torch.float32)
test_X_tensor = torch.tensor(test_X, dtype=torch.float32)
test_Y_tensor = torch.tensor(test_Y.reshape(-1, 1), dtype=torch.float32)

train_dataset = TensorDataset(train_X_tensor, train_Y_tensor)
test_dataset = TensorDataset(test_X_tensor, test_Y_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model_lstm = LSTMModel(input_size=num_features, hidden_size=num_hidden, num_layers=1, output_size=num_output).to(device)

optimizer_lstm = torch.optim.Adam(model_lstm.parameters(), lr=learning_rate)
criterion_lstm = nn.MSELoss()

train_loss_lst = []
test_loss_lst = []

max_epochs= 100

max_epochs = 100

for epoch in tqdm(range(max_epochs), desc="Training Epochs"):  # tqdm 사용
    train_loss = train(model_lstm, train_loader, optimizer_lstm, criterion_lstm)
    test_loss, actuals, predictions = validate_model(model_lstm, test_loader, criterion_lstm)
    train_loss_lst.append(train_loss)
    test_loss_lst.append(test_loss)

    if (epoch + 1) % 10 == 0:
        print(f"\nepoch {epoch+1}: train loss(mse) = {train_loss:.4f} test loss(mse) = {test_loss:.4f}")

        print(f"학습 완료 : 총 {epoch+1} epoch")

Training Epochs:  10%|██▌                      | 10/100 [01:11<10:42,  7.14s/it]


epoch 10: train loss(mse) = 0.1373 test loss(mse) = 0.1653
학습 완료 : 총 10 epoch


Training Epochs:  20%|█████                    | 20/100 [02:21<09:16,  6.96s/it]


epoch 20: train loss(mse) = 0.1367 test loss(mse) = 0.1652
학습 완료 : 총 20 epoch


Training Epochs:  30%|███████▌                 | 30/100 [03:31<08:13,  7.05s/it]

...


epoch 80: train loss(mse) = 0.1354 test loss(mse) = 0.1628
학습 완료 : 총 80 epoch


Training Epochs:  90%|██████████████████████▌  | 90/100 [10:42<01:14,  7.40s/it]


epoch 90: train loss(mse) = 0.1353 test loss(mse) = 0.1625
학습 완료 : 총 90 epoch


Training Epochs: 100%|████████████████████████| 100/100 [11:56<00:00,  7.17s/it]


epoch 100: train loss(mse) = 0.1352 test loss(mse) = 0.1629
학습 완료 : 총 100 epoch
